In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2
import torch
from transformers import GPTNeoForSequenceClassification, GPTNeoForCausalLM, GPT2Tokenizer
from model import GPTNeoForSequenceClassificationBinary
import lightning.pytorch as pl
from model import GPTSequenceClassifiationModule

In [3]:


test_name = 'data/ccs_test.csv' #'data/wordnet_all_false_july4_test.csv' #'data/ccs_test.csv'
actual_tree = 'data/acm_ccs_clean.csv' #'data/bansal_wordnet_true_pairs.csv' #'data/acm_ccs_clean.csv'
model_path = '/root/autodl-tmp/runs/ccs/finetune.ckpt'
save_path = 'results/ccs/random.csv'

model = GPTSequenceClassifiationModule.load_from_checkpoint(model_path)
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

Some weights of GPTNeoForSequenceClassification were not initialized from the model checkpoint at EleutherAI/gpt-neo-1.3B and are newly initialized: ['transformer.h.1.attn.attention.bias', 'transformer.h.23.attn.attention.bias', 'transformer.h.15.attn.attention.bias', 'transformer.h.19.attn.attention.bias', 'transformer.h.9.attn.attention.bias', 'transformer.h.21.attn.attention.bias', 'score.weight', 'transformer.h.11.attn.attention.bias', 'transformer.h.5.attn.attention.bias', 'transformer.h.17.attn.attention.bias', 'transformer.h.7.attn.attention.bias', 'transformer.h.13.attn.attention.bias', 'transformer.h.3.attn.attention.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPTNeoForSequenceClassification(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPTNeoBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=2048, out_features=8192, bias=True)
     

In [111]:
# randomly initialize the model
def random_init(model):
    num_layers = len(model.model.transformer.h)
    learnable_layers = model.hparams.learnable_layers if model.hparams.learnable_layers != -1 else num_layers
    for i in range(learnable_layers):
        print('Initializing layer ', num_layers - 1 - i)
        model.model.transformer.h[num_layers - 1 - i].mlp.apply(model.model._init_weights)
    model.model.score.apply(model.model._init_weights)

random_initialize_model = False
if random_initialize_model:
    random_init(model)

Initializing layer  23
Initializing layer  22
Initializing layer  21
Initializing layer  20
Initializing layer  19


In [112]:

model=model.eval()
model=model.cuda()

In [113]:
# test_name='/root/Taxonomy-GPT/data/wordnet_all_false_july4_test.csv'
# actual_tree='/root/Taxonomy-GPT/data/bansal_wordnet_true_pairs.csv'
# test_name='/root/Taxonomy-GPT/data/ccs_test.csv'
# actual_tree='/root/Taxonomy-GPT/data/acm_ccs_clean.csv'

In [114]:
from dataset import get_taxonomy_dataset, get_taxonomy_dataset_binary
dataset = get_taxonomy_dataset_binary(test_name, entire_dataset=True,remove_columns=True)

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-4c1478a641ccb039/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-4c1478a641ccb039/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-63444c9d80fd9331.arrow


In [115]:
dataset

{'test': Dataset({
     features: ['flag', 'input_ids', 'labels'],
     num_rows: 21800
 })}

In [116]:
def format_dataset(example,tokenizer):
    prompt_template = "I am doing the taxonomy research. I think {child} is a subtopic of {parent}"
    prompt = prompt_template.format(child=example['child'], parent=example['parent'])
    example['input_ids'] = tokenizer.encode(prompt)
    return example

def test_dataloader(str_path,batch_size):
    dataset= get_taxonomy_dataset_binary(str_path,entire_dataset=True)
    dataset = dataset['test']
    dataset = dataset.map(lambda example: format_dataset(example,tokenizer))
    dataset = dataset.remove_columns(['child', 'parent', 'group'])

    return torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=DataCollatorWithPadding(tokenizer),
        num_workers=4,
    )

In [117]:
from torch.utils.data import DataLoader
from transformers.data.data_collator import DataCollatorWithPadding
from tqdm.notebook import tqdm
import torch 

results = []
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorWithPadding(tokenizer)

testdataloader = test_dataloader(test_name, 64)
with torch.no_grad():
    for step, batch in enumerate(tqdm(testdataloader)):
        batch = batch.to('cuda')
        output = model(batch['input_ids'], batch['attention_mask'])
        predicted_labels = torch.sigmoid(output)
        results.extend(predicted_labels.cpu().detach().numpy().tolist())

Found cached dataset csv (/root/.cache/huggingface/datasets/csv/default-4c1478a641ccb039/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-4c1478a641ccb039/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-63444c9d80fd9331.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-4c1478a641ccb039/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-8e4487ffbe6f9edf.arrow


  0%|          | 0/341 [00:00<?, ?it/s]

In [118]:
import pandas as pd
df = pd.read_csv(test_name)
df['predict'] = results
df['pred_flag']=[True if i>0.5 else False for i in df['predict']]

In [119]:
df.to_csv(save_path, index=False)

In [120]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
y_test=df['flag']
y_pred=df['pred_flag']
print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))
print(accuracy_score(y_test, y_pred))

0.019967733678225963
0.5101385448206257
0.5000468208633767
0.02036697247706422


In [121]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       1.00      0.00      0.00     21358
        True       0.02      1.00      0.04       442

    accuracy                           0.02     21800
   macro avg       0.51      0.50      0.02     21800
weighted avg       0.98      0.02      0.00     21800



## Post processing

In [122]:
from utils import (
    convert_to_ancestor_graph, 
    maximum_absorbance, 
    dataframe_to_ancestor_graph, 
    evaluate_groups,
    maximum_likelihood
)

from tqdm import tqdm
import numpy as np


In [155]:
save_path = 'results/ccs/random.csv' # 'results/wordnet/lora.csv'
actual_tree = 'data/acm_ccs_clean.csv' # 'data/bansal_wordnet_true_pairs.csv'
post_process = maximum_absorbance #maximum_likelihood # maximum_likelihood maximum_absorbance

df_t = pd.read_csv(save_path)

In [156]:
df_t['predict'] = np.log(df_t['predict'])

In [157]:
df_t['predict']

0       -0.203009
1       -0.195474
2       -0.183556
3       -0.209004
4       -0.184365
           ...   
21795   -0.191086
21796   -0.191347
21797   -0.198790
21798   -0.189187
21799   -0.194152
Name: predict, Length: 21800, dtype: float64

In [158]:
forest=[]
for g in tqdm(list(set(df_t.group))):
    df = post_process(df_t, g)
    forest.append(df)

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [00:10<00:00,  1.37it/s]


In [159]:
res_v2=pd.concat(forest, ignore_index=True)
res_v2=dataframe_to_ancestor_graph(res_v2)

100%|██████████| 15/15 [00:00<00:00, 456.72it/s]


In [160]:
res_v2

,parent,child,group,compare
0,Software security engineering,Domain-specific security and privacy architect...,10002978_9,Software security engineeringDomain-specific s...
1,Software security engineering,Software and application security,10002978_9,Software security engineeringSoftware and appl...
2,Software security engineering,Social network security and privacy,10002978_9,Software security engineeringSocial network se...
3,Software and application security,Domain-specific security and privacy architect...,10002978_9,Software and application securityDomain-specif...
4,Software and application security,Social network security and privacy,10002978_9,Software and application securitySocial networ...
...,...,...,...,...
740,Signal processing systems,Beamforming,10010583_2,Signal processing systemsBeamforming10010583_2
741,Signal processing systems,Sensor applications and deployments,10010583_2,Signal processing systemsSensor applications a...
742,Signal processing systems,External storage,10010583_2,Signal processing systemsExternal storage10010...
743,Signal processing systems,Electro-mechanical devices,10010583_2,Signal processing systemsElectro-mechanical de...


In [161]:
res_v2['group'].value_counts()

group
10002951_5     182
10010147_3     156
10003456_2     102
10002950_2      64
10010583_10     46
10010583_7      33
10010147_5      30
10003120_3      26
10010405_4      22
10002978_3      20
10010583_8      20
10010583_2      20
10002950_5      11
10002978_9      10
10002950_3       3
Name: count, dtype: int64

In [162]:
df_actual=pd.read_csv(actual_tree)
df_actual=df_actual[df_actual['type'] == 'test']
df_actual['compare']=df_actual['parent']+df_actual['child']+df_actual['group'].astype(str)

In [163]:
df_actual.group

23      10002950_2
24      10002950_2
25      10002950_2
26      10002950_2
27      10002950_2
           ...    
1838    10003120_3
1839    10003120_3
1840    10003120_3
1841    10003120_3
1842    10003120_3
Name: group, Length: 442, dtype: object

In [164]:
df_actual = dataframe_to_ancestor_graph(df_actual)
df_actual['group'].value_counts()

100%|██████████| 15/15 [00:00<00:00, 457.63it/s]


group
10010147_3     186
10002951_5     145
10002950_2     110
10003456_2      98
10010147_5      55
10010583_10     51
10003120_3      45
10010405_4      35
10010583_7      33
10010583_2      25
10010583_8      15
10002950_5      15
10002978_3      12
10002978_9       5
10002950_3       3
Name: count, dtype: int64

In [165]:
df_actual[df_actual.group=='10010147_3']

,parent,child,group,compare
509,Artificial intelligence,Discrete space search,10010147_3,Artificial intelligenceDiscrete space search10...
510,Artificial intelligence,Matching,10010147_3,Artificial intelligenceMatching10010147_3
511,Artificial intelligence,Planning with abstraction and generalization,10010147_3,Artificial intelligencePlanning with abstracti...
512,Artificial intelligence,Interest point and salient region detections,10010147_3,Artificial intelligenceInterest point and sali...
513,Artificial intelligence,Robotic planning,10010147_3,Artificial intelligenceRobotic planning10010147_3
...,...,...,...,...
690,Natural language processing,Phonology / morphology,10010147_3,Natural language processingPhonology / morphol...
691,Philosophical/theoretical foundations of artif...,Cognitive science,10010147_3,Philosophical/theoretical foundations of artif...
692,Philosophical/theoretical foundations of artif...,Theory of mind,10010147_3,Philosophical/theoretical foundations of artif...
693,Control methods,Motion path planning,10010147_3,Control methodsMotion path planning10010147_3


In [166]:
res_v2[res_v2.group=='10010147_3']

,parent,child,group,compare
539,Probabilistic reasoning,Discrete space search,10010147_3,Probabilistic reasoningDiscrete space search10...
540,Probabilistic reasoning,Matching,10010147_3,Probabilistic reasoningMatching10010147_3
541,Probabilistic reasoning,Planning with abstraction and generalization,10010147_3,Probabilistic reasoningPlanning with abstracti...
542,Probabilistic reasoning,Artificial intelligence,10010147_3,Probabilistic reasoningArtificial intelligence...
543,Probabilistic reasoning,Interest point and salient region detections,10010147_3,Probabilistic reasoningInterest point and sali...
...,...,...,...,...
690,Randomized search,Phonology / morphology,10010147_3,Randomized searchPhonology / morphology10010147_3
691,Randomized search,Theory of mind,10010147_3,Randomized searchTheory of mind10010147_3
692,Planning under uncertainty,Tracking,10010147_3,Planning under uncertaintyTracking10010147_3
693,Planning under uncertainty,Search methodologies,10010147_3,Planning under uncertaintySearch methodologies...


In [167]:
recall, precision, f1 = evaluate_groups(df_actual, res_v2)

100%|██████████| 15/15 [00:00<00:00, 394.33it/s]


In [168]:
recall

0.17443432294547911

In [169]:
precision

0.162093908506952

In [170]:
f1

0.1607280989176318

In [35]:
df_common = pd.merge(df_actual, res_v2, on=['compare'], how='inner')

In [36]:
len(df_common)/len(df_actual)

0.49339735894357745

In [37]:
len(df_common)/len(res_v2)

0.2180371352785146

In [38]:
recall = []
precision = []
f1 = []
for group in tqdm(list(set(df_actual.group))):
    group_actual = df_actual[df_actual.group == group]
    group_pred = res_v2[res_v2.group == group]
    recall.append(len(group_actual.merge(group_pred, on='compare')) / len(group_actual))
    precision.append(len(group_actual.merge(group_pred, on='compare')) / len(group_pred))
    if precision[-1] + recall[-1] == 0:
        f1.append(0)
    else:
        f1.append(2 * (precision[-1] * recall[-1]) / (precision[-1] + recall[-1]))

  7%|▋         | 1/15 [00:00<00:00, 93.23it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 │   group_actual = df_actual[df_actual.group == group]                                      │
│    6 │   group_pred = res_v2[res_v2.group == group]                                              │
│    7 │   recall.append(len(group_actual.merge(group_pred, on='compare')) / len(group_actual))    │
│ ❱  8 │   precision.append(len(group_actual.merge(group_pred, on='compare')) / len(group_pred)    │
│    9 │   if precision[-1] + recall[-1] == 0:                                                     │
│   10 │   │   f1.append(0)                                                                        │
│   11 │   else:                                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ZeroDivisionError: division by zero

In [39]:
import numpy as np
np.mean(recall)

0.5

In [83]:
np.mean(precision)

0.6256533225972404

In [84]:
np.mean(f1)

0.550320877862728